## 描述性统计分析

In [1]:
# 忽略警告信息
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
student_info=pd.read_csv(r"D:\Entertainment\Heaven Pool\education_data\2_student_info.csv")
chengji=pd.read_csv(r"D:\Entertainment\Heaven Pool\education_data\5_chengji.csv")
examtype=pd.read_csv(r"D:\Entertainment\Heaven Pool\education_data\6_exam_type.csv")
# 当前在校学生信息
student_info.columns=[u'学生id',u'学生姓名',u'性别',u'民族',u'出生日期',u'班级名',u'家庭住址',u'家庭类型',u'政治面貌',u'班级id',u'班级学期',u'是否住校',u'是否退学',u'宿舍号']

# 学生成绩
chengji.columns=['考试id','考试编号','考试编码名称','考试学科id','考试学科名','考试学期','考试类型','考试开始时间','学生id','考试成绩','Z_score','T_score','等第']
# 使用pandas的to_datatime函数转换为标准日期格式

# 考试类型
examtype.columns=['考试类型id','考试类型名称']

In [80]:
# 合并chengji和student_info
info=pd.merge(student_info,chengji,on='学生id')
info=info[['学生id','学生姓名','班级名','班级id','考试学科名','考试学期','考试编码名称','考试类型','考试成绩','Z_score','T_score','等第']]
info.head()

,学生id,学生姓名,班级名,班级id,考试学科名,考试学期,考试编码名称,考试类型,考试成绩,Z_score,T_score,等第
0,14454,陈某某,白-高二(01),901,NaN,2018-2019-1,2018-1学期期末考试,3,83.0,NaN,NaN,NaN
1,14454,陈某某,白-高二(01),901,NaN,2018-2019-1,2018-1学期期末考试,3,55.0,NaN,NaN,NaN
2,14454,陈某某,白-高二(01),901,NaN,2018-2019-1,2018-1学期期末考试,3,78.0,NaN,NaN,NaN
3,14454,陈某某,白-高二(01),901,NaN,2018-2019-1,2018-1学期期末考试,3,67.0,NaN,NaN,NaN
4,14454,陈某某,白-高二(01),901,NaN,2018-2019-1,2018-1学期期末考试,3,110.0,NaN,NaN,NaN


In [142]:
# 删除缺考学生
special=info.ix[info['考试成绩']==-2]
info=info[-info['考试成绩'].isin(special['考试成绩'])]

### 1.1 各班级学生排名情况

#### 1.1.1  总分排名

In [143]:
# 按班级和考试名称筛选
Class=info.ix[(info['班级名']=='白-高二(01)') &  (info['考试编码名称']=='2018-1学期期末考试')]
Score=Class.groupby(['学生id','学生姓名'])['考试成绩'].sum()

In [144]:
# 整体班级和考试名称筛选
TEST=info.ix[info['考试编码名称']=='2018-1学期期末考试']
SCORE=TEST.groupby(['班级名','学生id','学生姓名'])['考试成绩'].sum().groupby(level=0, group_keys=False).nlargest(60)  
# 在index的第1个level进行group；不设置group_keys=False, 会有2列job；nlargest()函数：对每一个group进行排序，并且只取每group的前3个

# SCORE=SCORE.reset_index()
SCORE.to_excel('D:\Entertainment\Heaven Pool\\2018年第1学期期末各班总分排名.xls')

#### 1.1.2  单科排名

In [145]:
# 单科逐一筛选
TEST1=info.ix[info['考试编码名称']=='2018-1学期期中考试']
SUBJECT1=TEST1.ix[TEST1['考试学科名']=='数学']
SCORE1=SUBJECT1.groupby(['班级名','学生id','学生姓名'])['考试成绩'].sum().groupby(level=0, group_keys=False).nlargest(60)
SCORE1.to_excel('D:\Entertainment\Heaven Pool\\2018年第1学期期中数学班级排名.xls')

In [146]:
# 数据透视表
TEST2=info.ix[info['考试编码名称']=='2018-1学期期中考试']
SCORE2=pd.pivot_table(TEST,index=['班级名','学生id','学生姓名'],values=['考试成绩'],columns=['考试学科名'])
SCORE2=SCORE2.reset_index()
#SCORE.dtypes
SCORE2.to_excel('D:\Entertainment\Heaven Pool\\2018年第1学期期中考试各科成绩.xls')

### 1.2 各班级每个分数段学生排名人数

#### 1.2.1 各班各科分数段排名

In [152]:
TEST3=info.ix[info['考试编码名称']=='2018-1学期期中考试']
bins=[10,20,30,40,50,60,70,80,90,100,110,120,130,140,150]
TEST3['分数分组']=pd.cut(TEST3['考试成绩'],bins,right=False)
SCORE3=TEST3.groupby(['班级名','考试学科名','分数分组'])['学生id'].count().groupby(level=[0,1], group_keys=False).nlargest(100)
SCORE3.to_excel('D:\Entertainment\Heaven Pool\\2018年第1学期期中各班各科分数段排名.xls')

#### 1.2.2 各班总分分数段排名

In [148]:
TEST4=info.ix[info['考试编码名称']=='2018-1学期期中考试']
SUM=TEST4.groupby(['班级名','学生id'])['考试成绩'].sum()
SUM=SUM.reset_index()
bins=[100,200,300,400,500,600,700,800,900,1000]
SUM['分数分组']=pd.cut(SUM['考试成绩'],bins,right=False)
SCORE4=SUM.groupby(['班级名','分数分组'])['学生id'].count().groupby(level=0, group_keys=False).nlargest(400)
SCORE4.to_excel('D:\Entertainment\Heaven Pool\\2018年第1学期期中各班总分段排名.xls')

#### 1.2.3 各班Z-score分数段排名（Z-score 标准分）

In [159]:
TEST5=info.ix[info['考试编码名称']=='2018-1学期期中考试']  # -4.6 到2.9
bins=[-4.6,-4.0,-3.5,-3.0,-2.5,-2.0,-1.5,-1.0,-0.5,0,0.5,1.0,1.5,2.0,2.5,3.0]
TEST5['Z_score分组']=pd.cut(TEST5['Z_score'],bins,right=False)

SCORE5=TEST5.groupby(['班级名','考试学科名','Z_score分组'])['学生id'].count().groupby(level=[0,1], group_keys=False).nlargest(100)
SCORE5.to_excel('D:\Entertainment\Heaven Pool\\2018年第1学期期中各科Z_score分数段排名.xls')

#### 1.2.4  各班T_score分数段排名

In [162]:
TEST6=info.ix[info['考试编码名称']=='2018-1学期期中考试']  # -34 到103
bins=[-34,-30,-20,-10,0,10,20,30,40,50,60,70,80,90,105]
TEST6['T_score分组']=pd.cut(TEST6['T_score'],bins,right=False)
SCORE6=TEST6.groupby(['班级名','考试学科名','T_score分组'])['学生id'].count().groupby(level=[0,1], group_keys=False).nlargest(100)
SCORE6.to_excel('D:\Entertainment\Heaven Pool\\2018年第1学期期中各科T_score分数段排名.xls')

### 1.3  各个科目每个分数段学生排名

In [172]:
info['年级名']=info['班级名']
info['年级名'][info['年级名'].str.contains('高一')]=u'高一'
info['年级名'][info['年级名'].str.contains('高二')]=u'高二'
info['年级名'][info['年级名'].str.contains('高三')]=u'高三'
TEST7=info.ix[info['考试编码名称']=='2018-1学期期中考试']  # -34 到103
TEST7=TEST7.groupby(['年级名','考试学科名','学生id','学生姓名'])['考试成绩'].mean()
TEST7=TEST7.reset_index()
bins=[0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150]
TEST7['分数分组']=pd.cut(TEST7['考试成绩'],bins,right=False)
SCORE7=TEST7.groupby(['年级名','考试学科名','分数分组'])['学生id'].count().groupby(level=[0,1], group_keys=False).nlargest(100)
SCORE7.to_excel('D:\Entertainment\Heaven Pool\\2018年第1学期期中各科目分数段排名.xls')

### 数据透视表：https://www.cnblogs.com/stream886/p/6022125.html#q8

### Pandas中DataFrame基本操作（一），基本函数整理：https://blog.csdn.net/hhtnan/article/details/80080240